# 02. Limpieza de datos

Este notebook contiene la limpieza del dataset de víctimas del conflicto armado en Colombia.

## Importación de Librerías

Importamos las librerías necesarias para el análisis de datos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración general
sns.set_theme(style="whitegrid", font_scale=1.2)  # Estilo seaborn

# Opcional: puedes usar estilo matplotlib también (se combinan bien)
plt.style.use('ggplot')  # Cambia por 'fivethirtyeight', 'dark_background', etc. si quieres

# Otras configuraciones
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.grid'] = True

# Ver los estilos disponibles (por si quieres cambiar)
print(plt.style.available)


['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


## Carga del Dataset

Cargamos el dataset de víctimas del conflicto armado desde el archivo CSV.

In [13]:
# Define the relative path to the data file
data_path = '../data/raw/victimas_por_hechos_departamental_20250416.csv'

# Load the dataset
try:
    # Attempt to read with UTF-8 encoding first
    df = pd.read_csv(data_path, encoding='utf-8') 
except UnicodeDecodeError:
    try:
        # Fallback to Latin-1 encoding if UTF-8 fails
        df = pd.read_csv(data_path, encoding='latin1') 
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        df = None # Indicate failure by setting df to None

# Verificar si el dataset se cargó correctamente
if df is not None:
    print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
else:
    print("Error al cargar el dataset. Por favor, verifica la ruta y la codificación del archivo.")

Dataset cargado exitosamente. Dimensiones: (2431164, 17)


# Eliminacion de columnas innecesarias
Se eliminaran las columnas que no sean necesarias para el analisis como NOM_RPT, COD_PAI y PAIS

In [ ]:
# Mostrar forma inicial del dataframe y vista previa
print(f"Forma inicial del dataframe: {df.shape}")
print(f"Columnas del dataframe: {df.columns.tolist()}")

Forma inicial del dataframe: (2431164, 17)
Columnas del dataframe: ['FECHA_CORTE', 'NOM_RPT', 'COD_PAIS', 'PAIS', 'COD_ESTADO_DEPTO', 'ESTADO_DEPTO', 'PARAM_HECHO', 'HECHO', 'SEXO', 'ETNIA', 'DISCAPACIDAD', 'CICLO_VITAL', 'PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']


In [19]:
# Columnas que se eliminarán
columns_to_drop = ["NOM_RPT", "COD_PAIS", "PAIS", "COD_ESTADO_DEPTO"]

# Eliminar columnas especificadas
df = df.drop(columns=columns_to_drop)

In [20]:
# Mostrar forma final del dataframe
print(f"Forma final del dataframe después de eliminar columnas: {df.shape}")
print(f"Columnas del dataframe: {df.columns.tolist()}")
df.head()

Forma final del dataframe después de eliminar columnas: (2431164, 13)
Columnas del dataframe: ['FECHA_CORTE', 'ESTADO_DEPTO', 'PARAM_HECHO', 'HECHO', 'SEXO', 'ETNIA', 'DISCAPACIDAD', 'CICLO_VITAL', 'PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']


,FECHA_CORTE,ESTADO_DEPTO,PARAM_HECHO,HECHO,SEXO,ETNIA,DISCAPACIDAD,CICLO_VITAL,PER_OCU,PER_DECLA,PER_UBIC,PER_SA,EVENTOS
0,2022/03/31 00:00:00.000000000,Bolivar,5,Desplazamiento forzado,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 18 y 28,3.0,4.0,3.0,3.0,3.0
1,2022/02/28 00:00:00.000000000,Cesar,5,Desplazamiento forzado,Mujer,Raizal del Archipielago de San Andres y Provid...,Multiple,entre 61 y 100,1.0,NaN,NaN,NaN,1.0
2,2022/04/30 00:00:00.000000000,Caqueta,5,Desplazamiento forzado,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 29 y 60,2.0,NaN,NaN,NaN,2.0
3,2022/05/31 00:00:00.000000000,Guaviare,5,Desplazamiento forzado,Hombre,Negro(a) o Afrocolombiano(a),Fisica,entre 61 y 100,11.0,7.0,4.0,4.0,12.0
4,2022/04/30 00:00:00.000000000,Caqueta,2,Amenaza,Hombre,Indigena (Acreditado RA),Por Establecer,entre 12 y 17,1.0,1.0,1.0,1.0,1.0


# Corrección de errores de codificación y unificación de categorías

Se identificaron y reemplazaron valores con errores de codificación en varias columnas, con el fin de mejorar la calidad y consistencia de los datos. A continuación, se detallan los ajustes realizados:

### Columna: `ESTADO_DEPTO`

Valores corregidos:

* `"Nari�o"`
* `"NariÃ±o"`

Reemplazados por:

* `"Nariño"`

---

### Columna: `HECHO`

Valores corregidos:

* `"Desaparici�n forzada"`
* `"DesapariciÃ³n forzada"`

Reemplazados por:

* `"Desaparición forzada"`

---

* `"VinculaciÃ³n de NiÃ±os NiÃ±as y Adolescentes a Actividades Relacionadas con grupos armados"`
* `"Vinculaci�n de Ni�os Ni�as y Adolescentes a Actividades Relacionadas con grupos armados"`

Reemplazados por:

* `"Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados"`

---

* `"Minas Antipersonal, Munici�n sin Explotar y Artefacto Explosivo improvisado"`

Reemplazado por:

* `"Minas Antipersonal, Munición sin Explotar y Artefacto Explosivo improvisado"`

---

### Columna: `ETNIA`

Unificación de categorías para facilitar el análisis:

Valores originales:

* `"Negro (Acreditado RA)"`
* `"Afrocolombiano (Acreditado RA)"`

Reemplazados por:

* `"Negro(a) o Afrocolombiano(a) (Acreditado RA)"`


In [22]:
# Corrección de errores de codificación

df['ESTADO_DEPTO'] = df['ESTADO_DEPTO'].replace({
    'Nari�o': 'Nariño',
    'NariÃ±o': 'Nariño'
})

df['HECHO'] = df['HECHO'].replace({
    'Desaparici�n forzada': 'Desaparición forzada',
    'DesapariciÃ³n forzada': 'Desaparición forzada',
    'VinculaciÃ³n de NiÃ±os NiÃ±as y Adolescentes a Actividades Relacionadas con grupos armados':
        'Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados',
    'Vinculaci�n de Ni�os Ni�as y Adolescentes a Actividades Relacionadas con grupos armados':
        'Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados',
    'Minas Antipersonal, Munici�n sin Explotar y Artefacto Explosivo improvisado':
        'Minas Antipersonal, Munición sin Explotar y Artefacto Explosivo improvisado'
})

df['ETNIA'] = df['ETNIA'].replace({
    'Negro (Acreditado RA)': 'Negro(a) o Afrocolombiano(a) (Acreditado RA)',
    'Afrocolombiano (Acreditado RA)': 'Negro(a) o Afrocolombiano(a) (Acreditado RA)'
})


# Exportamos los datos limpios a un archivo CSV
Se exporta el archivo CSV con los datos limpios en en ´/data/processed/victimas_por_hechos_departamental_20250416.csv´ para su posterior análisis

In [ ]:
import csv
import os

df.to_csv(
    path_or_buf='../data/processed/victimas_por_hechos_departamental_20250416.csv',
       sep=',',
    na_rep='',
    header=True,
    index=False,
    encoding='utf-8',
    quoting=csv.QUOTE_MINIMAL,
    lineterminator=os.linesep,
    quotechar='"',
    decimal='.',
    errors='strict'
)

OSError: Cannot save file into a non-existent directory: 'data\processed'